## Location-Based Data Frame: Reshape data frame such that location and mutations are both represented in feature columns

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('amino_acid_genotypes_to_brightness.tsv', sep = '\t', engine = 'python')
df.head()

,aaMutations,uniqueBarcodes,medianBrightness,std
0,NaN,3645,3.719212,0.106992
1,SA108D,1,1.301030,NaN
2,SA108D:SN144D:SI186V:SM231T:SL234P,1,1.301031,NaN
3,SA108D:SY141N:SH146L:SE220G,1,1.301189,NaN
4,SA108G,2,3.708478,0.020613


In [40]:
mutants = df['aaMutations']       #split to list of aminoacids
mutants = mutants.str.split(':')
mutants = mutants[1:]
mutants.head()

1                                    [SA108D]
2    [SA108D, SN144D, SI186V, SM231T, SL234P]
3            [SA108D, SY141N, SH146L, SE220G]
4                                    [SA108G]
5            [SA108G, SK138M, SR166C, SL234P]
Name: aaMutations, dtype: object

Separating the 2ond entry to aaMut and position 

In [11]:
mutants[2]

['SA108D', 'SN144D', 'SI186V', 'SM231T', 'SL234P']

In [6]:
mut_separated = pd.Series(mutants).str.extract('(?P<position>^[A-Z]{2}\d+)(?P<mutation>[A-Z]$)')

mut_separated

C:\Users\Biel\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


,position,mutation
0,SA108,D
1,SN144,D
2,SI186,V
3,SM231,T
4,SL234,P


In [79]:
mut_position = []
for i in xrange(1,len(mutants)):
    mut_position.append(pd.Series(mutants[i]).str.extract('(?P<position>^[A-Z]{2}\d+)'))

C:\Users\Biel\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


In [80]:
df_mut_position = pd.DataFrame(mut_position,index=np.arange(len(mut_position)))

In [81]:
uniq_mut_pos=np.unique(df_mut_position.values) #number of columns 

In [82]:
df_mut_position.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,SA108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SA108,SN144,SI186,SM231,SL234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SA108,SY141,SH146,SE220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SA108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA108,SK138,SR166,SL234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
len(uniq_mut_pos)

234

In [84]:
uniq_mut_pos = uniq_mut_pos[1:]
len(uniq_mut_pos)

233

In [85]:
df_dummies = pd.get_dummies(df_mut_position, prefix='pos', drop_first=True)